In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import requests
import pandas as pd


class Alba():
    def __init__(self):

        self.flag = False
        self.result_list = []
        self.words = ["물류", "상차", "하차", "상하차", "투잡", "배달", "식기", 
                     "세척", "발렛", "쿠팡", "남여", "드라마", "품평회", "허브",
                     "생동성", "임상시험", "전화", "서빙", "엑스트라", "영화", 
                     "조리보조", "주방", "남녀", "고3", "피킹", "Picking", 
                     "냉장", "오렌지라이프", "누구나", "택배", "분류",
                      "연회장", "판매"]
        self.data = pd.DataFrame(columns=["title", "area", "pay", "time", "recently", "link"])

    def crawler(self, number_pages):

        for page in tqdm(range(1, number_pages+1)):
            req = requests.get('http://www.albamon.com/list/gi/mon_gi_list.asp?page=' +str(page)+'&gubun=2&ps=50&ob=6&lvtype=1&rArea=,I000&sDutyTerm=,5,10,20&gender=M&gender_chk=1&rAge=31&rAge_Chk=1&rWDate=1&Empmnt_Type=&tc=529')
            req.encoding = None
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')
            result_title = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.subject > div > p.cTit > a")
            result_area = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.area > div")
            result_pay = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.pay > p > img")
            result_recently = soup.select("#subcontent > form > div.gListWrap > table > tbody > tr > td.recently > em")

            for i in range(len(result_title)):
                self.data = self.data.append({"title" : result_title[i].text, 
                                            "area" : result_area[i].text, 
                                            "pay" : result_pay[i]['alt'],
                                            "time" : result_pay[i].findNext("td").text,
                                            "recently" : result_recently[i].text,
                                            "link" : "http://www.albamon.com/" + result_title[i]["href"]
                                            },
                                            ignore_index=True)
                    
        print("\n","crawling complete")


    def modifier(self):
        self.data["area"] = self.data["area"].apply(lambda x: x[6:-1] if True else x)
        self.data["time"] = self.data["time"].apply(lambda x: x[7:-7] if True else x)
        # self.data["recently"] = self.data["recently"].apply(lambda x: str(datetime.now().hour+9) + "시" + str(datetime.now().minute - int(x[:2])) + "분" if x[-2:] == "분전" and len(x) == 4 else x)
        # self.data["recently"] = self.data["recently"].apply(lambda x: str(datetime.now().hour+9) + "시" + str(datetime.now().minute - int(x[0])) + "분" if x[-2:] == "분전" and len(x) == 3 else x)
        # self.data["recently"] = self.data["recently"].apply(lambda x: str(datetime.now().hour+9 - int(x[0])) + "시" + str(datetime.now().minute)+ "분" if x[-2:] == "간전" else x)

        print("\n","modifier complete")

    def filter(self):
        drop_index_list = []
        for i in range(len(self.data)):
            for j in self.words:
                if j in self.data.title[i]:
                    drop_index_list.append(i)
                    continue
        self.data = self.data.drop(drop_index_list, axis=0)
        self.data = self.data.reset_index(drop=True)

        print("\n","filter complete")

alba = Alba()
alba.crawler(2)
alba.modifier()
alba.filter()
print(alba.data)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.35s/it]



 crawling complete

 modifier complete

 filter complete
                                   title    area pay         time recently  \
0   [최고단가] 기본일급 88,125원/ 6.5시간 근무/ 경력자.장  서울 관악구  일급  17:00~24:00     10분전   
1      [일일알바] 평가 아르바이트 패널 모집 (1시간/20000)  서울 전체   시급         시간협의     15분전   
2        장지역 (장기,단기,하루) 당일지급 홈쇼핑 상품 반품작업  서울 송파구  일급  09:00~18:00     16분전   
3                      연극 어플개발 관련 하루단기알바  서울 전체   시급         시간협의     18분전   
4         [단기근무/주부사원환영] 코스트코 양재점 가전제품 판촉  서울 서초구  일급  10:00~19:00     20분전   
5      [25일,26일,27일작업 3일지급]서울,분당선 실내무대사무  서울 전체   일급         시간협의     27분전   
6          강남역/꿀알바/DB손해보험 강남사업단에서 일일지점체험  서울 강남구  일급  09:00~12:00     30분전   
7        하루가능,선택근무가능/익일지급/70,240원/단순포장정리  서울 강동구  일급  09:00~18:00     42분전   
8         [단기가능/바로근무/익일지급가능/양천향교역부근 ] 가양  서울 강서구  시급         시간협의     42분전   
9         [단기가능/바로근무/익일지급가능/양천향교역부근 ] 가양  서울 강서구  시급         시간협의     42분전   
10        서울,경기권 집기 및 부업하실분 모집합니다(부업,단기,  서울 전체   일급  09:00~05:00     47분전   
11    